In [4]:
suppressPackageStartupMessages(library("affy"))
suppressPackageStartupMessages(library("affyio"))
suppressPackageStartupMessages(library("biomaRt"))
#remove.packages("GEOquery")
#library(devtools)
#install_github('seandavi/GEOquery',force = TRUE)
suppressPackageStartupMessages(library("GEOquery"))
suppressPackageStartupMessages(library("WGCNA"))

*
*  Package WGCNA 1.64.1 loaded.
*
*    Important note: It appears that your system supports multi-threading,
*    but it is not enabled within WGCNA in R. 
*    To allow multi-threading within WGCNA with all available cores, use 
*
*          allowWGCNAThreads()
*
*    within R. Use disableWGCNAThreads() to disable threading if necessary.
*    Alternatively, set the following environment variable on your system:
*
*          ALLOW_WGCNA_THREADS=<number_of_processors>
*
*    for example 
*
*          ALLOW_WGCNA_THREADS=4
*
*    To set the environment variable in linux bash shell, type 
*
*           export ALLOW_WGCNA_THREADS=4
*
*     before running R. Other operating systems or shells will
*     have a similar command to achieve the same aim.
*




# Clinical trials cohorts

###  GSE6434, GSE18864, GSE25055, GSE25065, GSE33072  
    - download raw CEL files via GEOquery
    - get array type and download corresponding CDF (probe layout)
    - read and preprocess intensities with justRMA()
    - annotate probe names with genes 
    - aggregate intensities to per-gene level
    - TODO: QC plots to compare intensity distributions and MA-plot; remove QC-failed samples
    
    
###  GSE9782 has two essential differences:
 1). Authors provided no CEL files:"Data was normalized to a Ttimmed mean of 15o and is NOT log transformed". Therefore, we only log2-transform the data. 
 
 2). For each sample two microarrays two chips available Affymetrix 133A and B. In this version we analyze them independently, although they designed to cover mostly different genes. We apply biomaRt to convert 



# GDSC-micro 

Raw data available at ArrayExpress E-MTAB-3610 https://www.ebi.ac.uk/arrayexpress/experiments/E-MTAB-3610/files/raw/
in 25 separate zip archives.

```bash
mkdir -p E-MTAB-3610;
cd E-MTAB-3610;
for i in `seq 1 25`; do
 wget https://www.ebi.ac.uk/arrayexpress/files/E-MTAB-3610/E-MTAB-3610.raw.$i.zip --quiet;
 unzip E-MTAB-3610.raw.$i.zip;
 rm E-MTAB-3610.raw.$i.zip;
done
cd .. ```

# CCLE-micro
* GSE36133 - the same processing pipeline as for clinical trial datasets 

### Results: 
 <data_root_dir>/preprocessed/exprs/<base_name>.justRMAlog2Average.EntrezGene.Expr.tsv
 
 OR
 
 <data_root_dir>/preprocessed/exprs/GSE9782.MAS5log2Average.EntrezGene.Expr.tsv
 
 
#### Tutorials:
https://github.com/obigriffith/biostar-tutorials/blob/master/MakeCdfEnv/MakeCdfEnv_example.R
https://github.com/icnn/Microarray-Tutorials/wiki/Affymetrix#2

In [18]:
###### set these variables: ######
# where to download raw data:
raw_data_root_dir <- "/home/olya/SFU/Hossein/arrays/"
# where to write preprocessed files 
preprocessed_data_dir <- "/home/olya/SFU/Hossein/v1/preprocessed/exprs/"
# whether to download CEL files; FALSE if *CEL.gz files are already in raw_data_root_dir/GEO_ID/
download_and_unpack <- TRUE
# BrainArray version; 22.0.0 is the latest; versions 17 and 15 were used in related works 
BrainArray_version <- '22.0.0'
for (dir in c(preprocessed_data_dir,raw_data_root_dir)){
    if (! file.exists(dir)){ 
        dir.create(file.path(dir),recursive =TRUE)
    }
}

In [4]:
myboxplot <- function(eset,title){
 options(repr.plot.width=10, repr.plot.height=4)
 title <- paste0(title, '/', annotation(eset))
 boxplot(exprs(eset), boxwex=0.6, notch=T, main=title, outline=FALSE, las=2)
 #legend("topleft", labels, fill=palette(),legend= c(1:7), bty="n")
}

myhist <- function(eset,title = "Histogram", xlabel = "RMAlog2"){
 i=1 
 plot( density((exprs(eset)[,i]),na.rm=T),main = title, xlab=xlabel)
 for(i in 2:dim(exprs(eset))[2]){
  lines(density((exprs(eset)[,i]),na.rm=T),)
 }
}

# convert files names to GSM sample names
getSnames <- function(sname,file_ext=".CEL.gz"){
    sname <- sub(file_ext, "", sname)
    sname <- strsplit(sname,split="_")
    sname <- unlist(sname)[1]
    return(sname)
}

getCDFname <- function(fname,dir=getwd()){
    cel_header <-affyio::read.celfile.header(paste0(dir,fname))
    cdfname <- cleancdfname(cel_header$cdfName)
    cdf_basename <- sub("cdf","",cdfname)
    return(cdf_basename)
}
downloadCELfromGEO <- function(GEO_id, raw_data_dir=NULL,download_and_unpack=TRUE,file_ext=".CEL.gz"){   
    if (is.null(raw_data_dir)) {
        raw_data_dir <- paste0(getwd(),GEO_id)
    }
    
    # download CEL files from GEO 
    if (download_and_unpack){
        if (! file.exists(raw_data_dir)){ 
            dir.create(file.path(raw_data_dir),recursive =TRUE)
        }
        message(c("Downloads raw CEL files using getGEOSuppFiles() into :",raw_data_dir))
        downloaded_files <- getGEOSuppFiles(GEO_id,makeDirectory = FALSE,baseDir = raw_data_dir)
        # untar 
        CELarchive <- paste0(raw_data_dir,GEO_id,"_RAW.tar")
        untar(CELarchive, exdir=raw_data_dir)
        # remove archive 
        file.remove(CELarchive)
    }
    # list CEL files and get GSM* sample names from file names
    fnames <- list.files(raw_data_dir ,pattern = file_ext)
    snames <- lapply(fnames, getSnames)
    snames <- unlist(snames)
    cdf_basenames <- unique(unlist(lapply(fnames,getCDFname, dir=raw_data_dir))) 
    return(list(raw_data_dir=raw_data_dir,sample_names=snames,fnames=fnames,cdfnames=cdf_basenames))    
}

# download and install BrainArray files 
getBrainArrayFile <-  function(chipname,what,version="22.0.0",gene_id_type="entrezg", download_dir=getwd()){
    fname <- paste0(chipname,gene_id_type,what,"_",version,".tar.gz")
    link <- paste0("http://mbni.org/customcdf/",version,"/",gene_id_type,".download/",fname)
    #print(link)
    fpath <- paste0(download_dir,fname)
    download.file(link, destfile = fpath, method = "wget")
    suppressWarnings(install.packages(fpath, repos = NULL, type="source"))
    libname <- paste0(chipname,gene_id_type,what)
    suppressPackageStartupMessages(library(libname,character.only = T))
    return(libname)
}
# for Ensembl:
# entrezg -> ensg
# .db -> probe
installBrainArrayCDFandDB <- function(chipname,version="22.0.0",gene_id_type="entrezg", download_dir=getwd()){
    if (chipname == "hugene10stv1hs"){
        chipname <- "hugene10sths"
        warning("hugene10stv1hs --> hugene10sths")
    }
    
    customCDFname <- getBrainArrayFile(chipname,"cdf",version=version,
                            gene_id_type=gene_id_type, download_dir=download_dir)
    # the same way for probe file
    
    customDBname <- getBrainArrayFile(chipname,".db",version=version,
                                   gene_id_type=gene_id_type, download_dir=download_dir)
    customProbe <- getBrainArrayFile(chipname,"probe",version=version,
                                   gene_id_type=gene_id_type, download_dir=download_dir)
    return (list(customCDFname=customCDFname, customDBname = customDBname,customProbe =customProbe ))
}


# map rownames of expression matrix to geneIDs, aggregate expressions to gene-level 
# and save .tsv files to the preprocessed_data_dir  
annotateAndWrite <- function(exprs_rma,geneID,customDBname,basename){
    anno <- select(get(customDBname), keytype="PROBEID", columns=c(geneID),
                   keys=row.names(exprs_rma))
    # drop ids unmapped on probes
    anno <- anno[!is.na(anno[,geneID]),]
    # aggregate expressions to gene level
    # actually, no CollapseRows() needed in case the mapping is one-to-one
    CR <- WGCNA::collapseRows(exprs_rma, rowGroup = anno[,geneID],rowID = anno[,"PROBEID"],method="Average")
    exprs_rma_gene_level <- CR$datETcollapsed

    outfile <- paste0(preprocessed_data_dir,basename,".BrainArray.RMAlog2Average.",geneID,".Expr.tsv")
    write.table(exprs_rma_gene_level, file =outfile,na="",quote = FALSE,sep="\t")
}
# MA-plot -- for QC


###  GSE6434, GSE18864, GSE25065, GSE33072  and GSE36133 (subset of GSE36139, CCLE)


In [5]:
for (GEO_ID in c("GSE36133","GSE33072","GSE25065","GSE18864","GSE6434")){ 
    basename <- GEO_ID
    raw_data_dir <- paste0(raw_data_root_dir,"/",GEO_ID,"/")

    raw_data <- downloadCELfromGEO(GEO_ID, raw_data_dir=raw_data_dir,
                                   download_and_unpack=download_and_unpack,file_ext=".CEL.gz")
    
    # download and install BrainArray CDF and annotations

    customAnno <- installBrainArrayCDFandDB(paste0(raw_data$cdfnames[1],"hs"),version=BrainArray_version,
                                            gene_id_type="entrezg",download_dir=raw_data_root_dir)
    # run RMA with custom CDF and obtain an 'ExpressionSet' object 
    
    rma <- affy::justRMA(celfile.path=raw_data$raw_data_dir,filenames = raw_data$fnames,
                                 sampleNames=raw_data$sample_names,verbose=FALSE,compress=FALSE,
                                 cdfname=customAnno$customCDFname)
    exprs_rma <- exprs(rma)


    # annotate probeset ids with names and save file to the preprocessed_data_dir
    for (geneID in c("ENTREZID","SYMBOL")){
        #print(geneID)
        annotateAndWrite(exprs_rma,geneID,customAnno$customDBname,basename)

    }
}



Downloads raw CEL files using getGEOSuppFiles() into :/home/olya/SFU/Hossein/arrays//GSE36133/
'select()' returned 1:1 mapping between keys and columns

'select()' returned 1:1 mapping between keys and columns
Downloads raw CEL files using getGEOSuppFiles() into :/home/olya/SFU/Hossein/arrays//GSE33072/
Warning message in installBrainArrayCDFandDB(paste0(raw_data$cdfnames[1], "hs"), :
“hugene10stv1hs --> hugene10sths”'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
Downloads raw CEL files using getGEOSuppFiles() into :/home/olya/SFU/Hossein/arrays//GSE25065/
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
Downloads raw CEL files using getGEOSuppFiles() into :/home/olya/SFU/Hossein/arrays//GSE18864/
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
Downloads raw CEL files using getGEOSuppFiles()

# The process is slightly different for GDSC, because it is absent in GEO 

Raw data available at ArrayExpress E-MTAB-3610 https://www.ebi.ac.uk/arrayexpress/experiments/E-MTAB-3610/files/raw/
in 25 separate zip archives.

(Assume .CEL files are already in the cel_file_dir )


In [6]:
basename <- "GDSC_micro"
# specify raw data directory
# replace this with download to raw_data_root_dir/E-MTAB-3610/
cel_file_dir = "/home/olya/SFU/Hossein/arrays/E-MTAB-3610/"


getSnames <- function(sname,ext=".cel"){
    sname <- sub(ext, "", sname)
    sname <- unlist(sname)[1]
    return(sname)
}
fnames <- list.files(cel_file_dir ,pattern = ".cel")
snames <- lapply(fnames, getSnames)
snames <- unlist(snames)
# number of CEL files:
length(snames)
#get CDF names from platform names from CEL headers 
getCDFname <- function(fname,dir=getwd()){
    cel_header <-affyio::read.celfile.header(paste0(dir,fname))
    return(cleancdfname(cel_header$cdfName))
}
cdf_basenames <- unique(unlist(lapply(fnames,getCDFname, dir=cel_file_dir)))
cdf_basenames 
# add processing of batches in future
cdf_basename <- sub("cdf","",cdf_basenames[1])

# load BrainArray
customAnno <- installBrainArrayCDFandDB(paste0(cdf_basename,"hs"),version=BrainArray_version,
                                        gene_id_type="entrezg",download_dir=raw_data_root_dir)
GDSC_rma <- affy::justRMA(celfile.path=cel_file_dir,filenames = fnames,sampleNames=snames,
                     verbose=FALSE,compress=FALSE,cdfname=customAnno$customCDFname)

GDSC_exprs <- exprs(GDSC_rma)
dim(GDSC_exprs)
# annotate probeset ids with names and save file to the preprocessed_data_dir
for (geneID in c("ENTREZID","SYMBOL")){
    annotateAndWrite(GDSC_exprs,geneID,customAnno$customDBname,basename)
}

[1] 1018

[1] "hgu219cdf"

[1] 18653  1018

'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns


### GSE9782

* no CEL files provided for this dataset
* two expression matrices for A and B arrays:
We use **biomaRt** to covert probeset IDs to Entrez gene ID
    - GPL96 - "affy_hg_u133a"
    - GPL97 - "affy_hg_u133b"

In [19]:
basename <- "GSE9782"
platform2biomaRt <- list("GPL96" = "affy_hg_u133a", "GPL97" = "affy_hg_u133b")

ensembl75 <- useMart(host='feb2014.archive.ensembl.org', 
                     biomart='ENSEMBL_MART_ENSEMBL', 
                     dataset='hsapiens_gene_ensembl')

tmp_file_dir <- paste0(raw_data_root_dir,basename)
if (! file.exists(tmp_file_dir)){ 
    dir.create(file.path(tmp_file_dir),recursive =TRUE)
}

esets <- suppressMessages(getGEO(basename,destdir = tmp_file_dir))
for (eset in esets){
    # log-transform expression
    exprs(eset) <-  log2(exprs(eset))
    
    # annotate and write file
    identifier <- platform2biomaRt[[annotation(eset)]]
    getinfo <- c(identifier, "entrezgene") #,'hgnc_symbol','external_gene_id')
    anno <- biomaRt::getBM(attributes = getinfo, filters=identifier, values = rownames(exprs(eset)),mart=ensembl75)
    anno <- anno[!is.na(anno$entrezgene),]
    # keep unique rows
    anno <- unique(anno)
    # drop probes mappe to momre than one gene
    anno <- anno[!duplicated(anno[,identifier]),]
    
    CR <- WGCNA::collapseRows(exprs(eset), rowGroup = anno$entrezgene, 
                   rowID = anno[,identifier],method="Average")
    write.table(CR$datETcollapsed, file = paste0(preprocessed_data_dir,basename,"-",annotation(eset),".MAS5log2Average.ENTREZID.Expr.tsv"),na="",quote = FALSE,sep="\t")
    #dim(CR$datETcollapsed)
    #head(CR$datETcollapsed)
}

In [3]:
sessionInfo()

R version 3.5.0 (2018-04-23)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 16.04.5 LTS

Matrix products: default
BLAS/LAPACK: /home/olya/miniconda2/lib/R/lib/libRblas.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=de_DE.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=de_DE.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=de_DE.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=de_DE.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] GEOquery_2.47.18    biomaRt_2.32.1      affyio_1.46.0      
[4] affy_1.54.0         Biobase_2.36.2      BiocGenerics_0.22.1

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.18          pillar_1.3.0          bindr_0.1.1          
 [4] BiocInstaller_1.26.1  compiler_3.5.0        bitops_1.0-6 